In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize,word_tokenize
import warnings

Using TensorFlow backend.


In [2]:
df=pd.read_csv('trainDatamodified.csv',encoding="ISO-8859-1")
df

,documentId,combinedDesc,category_groups_list,Data and Analytics,Hardware,Internet Services,Real Estate,Commerce and Shopping,Media and Entertainment,Transportation,...,Consumer Electronics,Education,Food and Beverage,Lending and Investments,Navigation and Mapping,Travel and Tourism,Agriculture and Farming,Sustainability,Natural Resources,Music and Audio
0,35440,enlighted is an iot platform for commercial re...,"Data and Analytics,Hardware,Internet Services,...",1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,13117,darrb is a platform that connects people who w...,"Commerce and Shopping,Internet Services,Media ...",0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,44816,grand perfecta is a leading provider of data a...,"Advertising,Events,Media and Entertainment,Sal...",0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4710,iapp me offers tools that enable its users to ...,"Apps,Information Technology,Mobile,Platforms,S...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26697,dealfaint provide top quality products and unp...,"Clothing and Apparel,Commerce and Shopping,Com...",0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,64676,affinityanswers enables consumer media brands ...,"Data and Analytics,Design,Information Technolo...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,62474,newchinacareer is a chinese online job board t...,"Administrative Services,Advertising,Internet S...",0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,12980,the most innovative global marketplace for art...,"Artificial Intelligence,Commerce and Shopping,...",1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,16454,vaximm a vaccine company based in switzerland ...,"Biotechnology,Health Care,Science and Engineer...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9504,the australian communications and media author...,"Government and Military,Hardware,Media and Ent...",0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.isna

<bound method DataFrame.isna of        documentId                                       combinedDesc  \
0           35440  enlighted is an iot platform for commercial re...   
1           13117  darrb is a platform that connects people who w...   
2           44816  grand perfecta is a leading provider of data a...   
3            4710  iapp me offers tools that enable its users to ...   
4           26697  dealfaint provide top quality products and unp...   
5           64676  affinityanswers enables consumer media brands ...   
6           62474  newchinacareer is a chinese online job board t...   
7           12980  the most innovative global marketplace for art...   
8           16454  vaximm a vaccine company based in switzerland ...   
9            9504  the australian communications and media author...   
10          69050  ingenious technologies ag is an independent te...   
11          77364  cheap panel smm is the perfect solution for pe...   
12          45389  guide it prof

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['combinedDesc'],df.drop(["documentId","combinedDesc","category_groups_list"],axis=1) , test_size=0.20, random_state=42)

In [5]:
X_test

7323     encon wastewater evaporator systems dramatical...
7003     the fashion communityyoudress aca a o the fash...
29534    calibration xrf cold chain standardsalternate ...
23677    vinny is an app that provides users with infor...
29695    the bunker secure hosting delivers secure mana...
15336    meridian offers a location based platform that...
22130    with m you can automatically invest in what yo...
17819    a single stop solution for all your digital ne...
15821    farallon geographics is a san francisco based ...
10424    aerospace parts locator service and database l...
216      bechdaalo is a free classified posting site in...
5961     ncn technology is a software development consu...
28989    tweetvisor is an interface for bloggers journa...
33664    replicate designs specializes in creating high...
30886    green products with eco friendly packaging and...
20296    award winning social media agency offering res...
34091    turbo hire delivers human like ai to automate .

In [16]:
import gensim
from gensim.models import Word2Vec

ModuleNotFoundError: No module named 'gensim'

In [6]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [9]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('glove.6B.300d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [14]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(46, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [15]:
history = model.fit(X_train, y_train, batch_size=128, epochs=7, verbose=1, validation_split=0.2)


Train on 26027 samples, validate on 6507 samples
Epoch 1/7
20864/26027 [=======================>......] - ETA: 20s - loss: 17.6328 - acc: 0.0209

KeyboardInterrupt: 

In [19]:
y_pred= model.predict(X_test)
#from sklearn.metrics import f1_score
#score = f1_score( y_test,y_pred,average='micro')

#print(score)
y_pred

array([[8.9902014e-02, 4.4970354e-01, 7.2854996e-02, ..., 5.8121890e-01,
        2.8628695e-01, 2.5446117e-03],
       [1.2783021e-02, 3.3217609e-02, 4.1274056e-01, ..., 1.0484755e-02,
        1.2918711e-03, 1.6375959e-02],
       [1.6270155e-01, 4.2825553e-01, 1.5155822e-02, ..., 3.8794607e-02,
        2.6133657e-02, 8.2463026e-04],
       ...,
       [2.3258680e-01, 3.7651747e-02, 3.6098212e-01, ..., 1.9214958e-02,
        6.1948299e-03, 1.0485321e-02],
       [8.8484019e-02, 1.5176237e-02, 6.2984264e-01, ..., 1.0340512e-03,
        2.5755167e-04, 3.1923383e-02],
       [5.1344872e-02, 1.6987219e-01, 3.0001551e-02, ..., 7.8228098e-01,
        7.2584212e-01, 3.8942993e-03]], dtype=float32)

# 

In [20]:
result = []

for sublist in y_pred:
    float_sublist = []
    for x in sublist:
        if x>=0.5:
            float_sublist.append(1)
        else:
            float_sublist.append(0)

    result.append(float_sublist)


In [21]:
result

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0],
 [0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
 

In [27]:
#from sklearn.metrics import f1_score
#score = f1_score( y_test,result,average='micro')

#print(score)
len(result[0])

46

In [162]:
test=pd.read_csv('testData.csv',encoding='ISO-8859-1')

In [163]:
test.fillna('a',inplace=True)

In [164]:
test.isna().sum()


Unnamed: 0      0
documentId      0
combinedDesc    0
dtype: int64

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

,Unnamed: 0,documentId,combinedDesc
0,0,53591,funmobility connects brands and audiences thro...
1,1,45826,ieveryware - ieveryware is a platform to devel...
2,2,76744,agterra provides data collection and mapping s...
3,3,51517,a leading provider of new and remanufactured e...
4,4,44883,underwrite.ai applies advances in artificial i...
5,5,60983,medical practice marketing ad agencyuse an exp...
6,6,71046,the executive centre provides serviced offices...
7,7,3780,digital health startup. smart sensor + app to ...
8,8,2782,emusic is a digital service that allows its su...
9,9,7105,los angeles based web design and development.i...


In [165]:
te=test['combinedDesc']
test = tokenizer.texts_to_sequences(te)
test = pad_sequences(test, padding='post', maxlen=maxlen)


In [166]:
y_pred=model.predict(test)

In [168]:
y_pred

array([[2.0014939e-01, 3.1469792e-02, 3.7309378e-01, ..., 4.0075183e-04,
        6.5475702e-05, 2.1562457e-02],
       [4.9580961e-02, 1.2922964e-01, 6.8850207e-01, ..., 3.6978722e-04,
        4.4912100e-05, 1.1902532e-01],
       [4.2972517e-01, 4.0833893e-01, 5.8940148e-01, ..., 2.9502213e-03,
        6.1088800e-04, 7.9199672e-04],
       ...,
       [6.6090107e-01, 2.2041491e-01, 1.1053133e-01, ..., 8.2836151e-03,
        3.7373304e-03, 5.8376789e-04],
       [3.9588392e-02, 8.0726689e-01, 8.0863059e-02, ..., 4.5521259e-03,
        2.5249720e-03, 3.4816563e-03],
       [1.2650281e-02, 3.6380911e-01, 6.6182405e-02, ..., 1.5051246e-02,
        6.4222813e-03, 1.5501112e-02]], dtype=float32)

In [170]:
y_pred.shape

(5000, 46)

In [171]:

result = []

for sublist in y_pred:
    float_sublist = []
    for x in sublist:
        if x>=0.5:
            float_sublist.append(1)
        else:
            float_sublist.append(0)

    result.append(float_sublist)

In [29]:
li=[]
for i in df.columns:
    if i=='documentId':
        continue
    elif i=='combinedDesc':
        continue
    elif i=='category_groups_list':
        continue
    else:
        li.append(i)
    

In [30]:
out = list()
for i in range(len(result)):
    x = result[i]
    l = list()
    
    for j in range(len(x)):
        if x[j]==1:
            l.append(li[j])
#     print(l)
    s = ''
    for k in range(len(l)):
        if k!=len(l)-1:
            s+=l[k]+','
        else:
            s+=l[k]
#     print(s)
    out.append(s)
print(out)

['Energy,Sustainability', 'Commerce and Shopping,Community and Lifestyle,Design', 'Science and Engineering,Biotechnology,Health Care,Manufacturing', 'Data and Analytics,Commerce and Shopping,Apps,Mobile,Software', 'Hardware,Information Technology,Software,Privacy and Security', 'Apps,Mobile,Software', 'Software,Financial Services,Privacy and Security,Lending and Investments', 'Internet Services,Advertising,Sales and Marketing,Design', 'Data and Analytics,Information Technology,Software,Privacy and Security', 'Software,Health Care', 'Internet Services,Commerce and Shopping,Media and Entertainment,Advertising,Sales and Marketing', 'Information Technology,Mobile,Software', 'Internet Services,Media and Entertainment,Software', 'Media and Entertainment,Design', 'Food and Beverage', 'Media and Entertainment,Advertising,Sales and Marketing,Software', 'Data and Analytics,Software,Artificial Intelligence,Science and Engineering', 'Internet Services,Commerce and Shopping,Media and Entertainment,

In [31]:
t=pd.read_csv('testData.csv',encoding='ISO-8859-1')
t['category'] = out
t = t[['documentId', 'category']]


t.iloc[26,:]


for i in range(len(t)):
    if len(t.iloc[i,1])==0:
        t.iloc[i,1]='no label'

t.to_csv('i.csv',index=False)

ValueError: Length of values does not match length of index

5000

['Media and Entertainment,Advertising,Sales and Marketing,Mobile,Software', 'Internet Services,Media and Entertainment,Apps,Mobile,Platforms,Software', 'Internet Services,Information Technology,Software', 'Hardware,Manufacturing,Consumer Electronics', 'Data and Analytics,Software,Artificial Intelligence,Science and Engineering,Financial Services', 'Advertising,Sales and Marketing', 'Hardware,Internet Services,Information Technology', 'Mobile,Software,Health Care', 'Internet Services,Media and Entertainment,Software', 'Internet Services,Commerce and Shopping,Design', 'Internet Services,Commerce and Shopping,Clothing and Apparel,Design', 'Hardware,Manufacturing,Consumer Electronics', 'Science and Engineering,Biotechnology,Health Care', 'Apps,Information Technology,Mobile,Software', 'Hardware,Software,Science and Engineering,Health Care', 'Software,Science and Engineering,Biotechnology,Health Care', 'Internet Services,Information Technology', 'Internet Services,Media and Entertainment,Com

,documentId,category
0,53591,"Media and Entertainment,Advertising,Sales and ..."
1,45826,"Internet Services,Media and Entertainment,Apps..."
2,76744,"Internet Services,Information Technology,Software"
3,51517,"Hardware,Manufacturing,Consumer Electronics"
4,44883,"Data and Analytics,Software,Artificial Intelli..."
5,60983,"Advertising,Sales and Marketing"
6,71046,"Hardware,Internet Services,Information Technology"
7,3780,"Mobile,Software,Health Care"
8,2782,"Internet Services,Media and Entertainment,Soft..."
9,7105,"Internet Services,Commerce and Shopping,Design"
